# A/B Test Data (Kaggle)

For this project there will be a single dataset that was downloaded from [Kaggle](https://www.kaggle.com/datasets/sergylog/ab-test-data), there is no specific information about how or where the data was obtained from. It is only stated that the data was from a site that launched an A/B test in order to increase income.

## 1.0 Discussing the dataset

The dataset only has 3 variables:

- *USER_ID*: it states the user ID of the user.
- *VARIANT_NAME*: it states if the user was part of the variant or control group.
- *REVENUE*: it states the revenue generated by the user.

## 2.0 Importing Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb

c:\Users\manue\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


## 3.0 Gathering the data

In [2]:
df = pd.read_csv('AB_Test_Results.csv')
df.head()

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


## 4.0 Assessing the data

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [132]:
df.nunique()

USER_ID         6324
VARIANT_NAME       2
REVENUE          101
dtype: int64

In [131]:
filter_list = []
known = {}
for index, row in df.iterrows():
    entry = known.get(row.USER_ID)
    if entry != None and entry != row.VARIANT_NAME:
        filter_list.append(row.USER_ID)
    known[row.USER_ID] = row.VARIANT_NAME
len(filter_list)

1829

### 4.1 Comments on Assessment
- Data is not missing any values.
- Variables have the correct data type.
- Several users (3676) repeated the test in the same or different group as previously.
  - Several users (1829) repeated the test at least once in both groups

## 5.0 Cleaning
In this section of the report, a cleaning of the dataset according to the findings during the assessment wil be made.

In [134]:
df_clean = df

### 5.1 Several users repeated the test at least once in both groups

#### 5.1.1 Define
For A/B testing, the fact that several users had repeated the test several times and for both test groups makes these user results invalid (especially since there is no indication of which one was done first)